# **INTRODUCTION TO PANDAS**
---
<img src="http://www.doc.ic.ac.uk/~afd/images/logo_imperial_college_london.png" align = "left" width=200>
 <br><br><br><br>
 
- Copyright (c) Antoine Jacquier, 2022. All rights reserved

- Author: Jack Jacquier <a.jacquier@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.9

 **Origins**
 
 `pandas` was originally developed by [Wes McKinney](https://wesmckinney.com) while at AQR Capital in 2007-2010.

# Introduction to DataFrames

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

## Create a Dataframe with dates

We start by creating series of Gaussian distributed values.

In [ ]:
N = 10000 ## size of the dataframe

In [ ]:
xx = np.random.standard_normal((N, 5))
df = pd.DataFrame(xx)

In [ ]:
type(df)

In [ ]:
df.head() ## Print the top five rows

In [ ]:
df.tail() ## Print the bottom five rows

In [ ]:
df.columns = ["Series 1", "Series 2", "Series 3", "Series 4", "Series 5"]
## Or more concisely:
#df.columns = ["Series %s" % i  for i in range(1,6)]
df.head()

Note that the index (first column on the left) has no real meaning here. 
However, in practice, thinking or each other column as time series, it may represent dates. 
We re-index the whole dataframe with this in mind.

#### Specifying a start date

In [ ]:
startYear = '2018'
startMonth = '01'
startDay = '01'
startDate = startYear + "-" + startMonth + "-" + startDay ## string concatenation
print("Start date: ", startDate)

In [ ]:
dates = pd.date_range(startDate, periods=N, freq="D")
df.index = dates
df.index.name = 'Date'
df.head()

## Basic dataframe manipulations

In [ ]:
df.keys()

An Index is a `pandas` class for an immutable sequence, which is the basis for axes.

In [ ]:
print('Names of columns:', [d for d in df.keys().values])
## similarly df.columns

Overview of the data

In [ ]:
df.info()

Basic statistics

In [ ]:
df.describe()

## Accessing elements from a dataframe

A  DataFrame may be indexed like an array, specifying the row and column number using .iloc (index location).
The location can be called either by the index number or by its name.

In [ ]:
df.head()

In [ ]:
print("First row, second column: ", df.iloc[0, 1], " or ", df.loc[startDate, 'Series 2'])

In [ ]:
myRow = df.iloc[2,:]
print(myRow)
print("Type: ", type(myRow))

Equivalently:

In [ ]:
myRow = df.loc['2018-01-03']
print(myRow, type(myRow))

In [ ]:
print("Accessing elements by slicing over columns: ")
print(df.iloc[0, 1:3])
print(type(df.iloc[0, 1:3]))

In [ ]:
print("Accessing elements by slicing over rows: ")
print(df.iloc[10:14, 1])
print(type(df.iloc[10:14, 1]))

In [ ]:
print("Accessing elements by slicing over boths: ")
print(df.iloc[10:14, 1:3])
print(type(df.iloc[10:14, 1:3]))

Question: What are the types of the following objects:
- df.iloc[2]
- df.iloc[2].values
- [d for d in df.iloc[2]]

In [ ]:
print(type(df.iloc[2]))
print(type(df.iloc[2].values))
print(type([d for d in df.iloc[2]]))

### `.at` and `.iat`

These two commands are very similar to `.loc` and `.iloc`, and are actually faster, but slicing is not permitted.

In [ ]:
%timeit df.iloc[10, 2]
%timeit df.iat[10, 2]

In [ ]:
df.iat[10, 2:4]

### Extract elements with conditions

In [ ]:
mySeriesName = "Series 2"
df[df[mySeriesName] > 3.4]

Note that the series `df` has not been modified. Only a copy has been printed!!

In [ ]:
mySeries =  df[mySeriesName][df[mySeriesName] > 3.4]
mySeries

We can return the series only as a list:

In [ ]:
print(mySeries.values)

### User-defined functions

In [ ]:
df = 2.*df
df.head()

Here, the original dataframe has been modified!

In [ ]:
def myFunction(x, param):
    return x + param

df[mySeriesName] = df[mySeriesName].apply(myFunction, args=(10,))
df.head()

## Numpy on dataframe

This is where the power of the DataFrame comes into the picture, namely that operations need to be performed element by element, 
but apply to a whole series (column) at once.

In [ ]:
myIndex = input("Series index")

In [ ]:
ser = 'Series ' + myIndex
print('Maximum on the first Series:', df[ser].max())
print('Date (index) where this maximum is attained:', df[ser].argmax()) ## deprecated
print('Date where this maximum is attained:', df[ser].idxmax(), df.index[df[ser].argmax()])
print('Mean of the first series:', df[ser].mean())

We can also obtain general results about the whole dataframe / each series in the dataframe:

In [ ]:
df.mean()

## Plotting Dataframes

##### Plot all the time series

In [ ]:
df.plot();

**WHAT IS WRONG ABOUT THE PLOT?**

##### Plot one column at a time

In [ ]:
df[ser].plot(legend=ser);

##### Changing the type of plots

In [ ]:
datePlot = '2018-01-03'
df.loc[datePlot].plot(kind='bar', title=datePlot); ## for one particular day

## Sorting DataFrames

In [ ]:
print('Data sorted by ' + ser)
df.sort_values(by=ser).head()

*WARNING: the dataframe df itself has remained unchanged though...*

In [ ]:
df.head()

## Renaming columns

In [ ]:
df.rename(columns={'Series 1': 'SPX', 
                   'Series 2': 'DAX', 
                   'Series 3': 'GOOGL', 
                   'Series 4': 'AAPL', 
                   'Series 5': 'VIX'}, inplace=True)
df.head()

### A note on copies on slices....

In [ ]:
df.head()

In [ ]:
val = df.iloc[1,0]
val

In [ ]:
df[df["SPX"] == val]["DAX"] = 0.
df

In [ ]:
df.loc[df["SPX"] == val, "DAX"] = 0.
df

# Working with real data

## Financial data from Yahoo Finance

In [ ]:
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web
from pandas_datareader import data as pdr
from datetime import datetime

In [ ]:
ticker = "AAPL"
aapl = pdr.DataReader(ticker, 'yahoo', '1980-01-01')
aapl.head()

In [ ]:
aapl.info()

In [ ]:
#aapl["Open"][1000:3000].plot(title=ticker)
aapl["Close"].plot(title=ticker)
plt.show()

## Economics Data from FRED

In [ ]:
from datetime import date
series_code = 'DGS10' # 10-year Treasury Rate
data_source = 'fred' # FED Economic Data Service
start = date(1962, 1, 1)
myData = pdr.DataReader(series_code, data_source, start)

In [ ]:
myData.head()

In [ ]:
myData = myData / 100.
series_name = '10-year Treasury'
myData = myData.rename(columns={series_code: series_name})
myData.plot(title=series_name)
plt.show() 

## Analysing real data

In [ ]:
ticker = "^GSPC" ## S&P500 
spx = pdr.DataReader(ticker, 'yahoo', '1980-01-01')
spx.info()

In [ ]:
spx.head()

In [ ]:
spx['Close'].plot(figsize=(8, 5), grid=True);

### A quick note on computation time -- computing daily returns

In [ ]:
len(spx)

In [ ]:
%%time
spx['ReturnsLoop'] = 0.
for i in range(1, len(spx)):
    spx['ReturnsLoop'][i] = np.log(spx['Close'][i] / spx['Close'][i - 1])

In [ ]:
spx[['Close', 'ReturnsLoop']].head()

In [ ]:
%time spx['Returns'] = np.log(spx['Close'] / spx['Close'].shift(1))
spx[['Close', 'Returns']].head()

In [ ]:
del spx['ReturnsLoop']

In [ ]:
spx.dropna()
spx.head()

In [ ]:
val = df.iloc[0,0]
val

In [ ]:
df[df['SPX'] == val]['DAX'] = 0.
df.head()

In [ ]:
df.loc[df['SPX'] == val, 'DAX'] = 0.
df.head()

**WHAT IS THE PROBLEM?**

### Data analysis

In [ ]:
spx[['Close', 'Returns']].plot(subplots=True, style='b',figsize=(8, 6));

**Remarks:**
- Volatility clustering
- Leverage effect

### Moving averages

In [ ]:
n1, n2 = 24, 252
MA1, MA2 = 'MA' + str(n1) + 'd', 'MA' + str(n2) + 'd'
spx[MA1] = spx['Close'].rolling(window=n1, center = False).mean()
spx[MA2] = spx['Close'].rolling(window=n2, center = False).mean()
spx.tail()

In [ ]:
spx[['Close', MA1, MA2]].plot(figsize=(8, 5), grid=True);

In [ ]:
spx['MovingReturnsVol'] = spx['Returns'].rolling(window=252).std() * np.sqrt(252.)

In [ ]:
spx[['Close', 'Returns', 'MovingReturnsVol']].plot(subplots=True, style='b',figsize=(8, 7), grid=True);

# The `groupby` command

In [ ]:
import pandas as pd

df = pd.read_excel('Class 2022-2023.xlsx')
df.head()

In [ ]:
df["Grade"].mean()

In [ ]:
df["Grade"].sum()# / len(df.index)

In [ ]:
df.groupby(['Nationality']).agg({'Grade': 'mean'})

In [ ]:
df.groupby(['Nationality']).agg({'Grade': 'sum'})

In [ ]:
df.groupby(['Nationality', 'Gender']).agg({'Grade': 'sum'}).sum()

**WHAT IS HAPPENING????**

# Merge, join, concatenate data -- an example with the MTA database

NYC map: https://www.google.com/maps/place/New+York,+NY,+USA/@40.6976684,-74.2605634,10z/data=!3m1!4b1!4m5!3m4!1s0x89c24fa5d33f083b:0xc80b8f06e177fe62!8m2!3d40.7127753!4d-74.0059728


Data: http://web.mta.info/developers/turnstile.html



### Importing and looking at the data

In [ ]:
import datetime as dt
import pandas as pd

In [ ]:
url  = r'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180825.txt'
df = pd.read_csv(url)#, index_col=0, header=0, parse_dates=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#!pip install xlrd

In [ ]:
url  = r'http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls'
infoStations = pd.read_excel(url)#, index_col=0, header=0, parse_dates=True)
infoStations.head()

### Cleaning the data

In [ ]:
df['ENTRIES'].max()

In [ ]:
df['EXITS'].max()

In [ ]:
df.keys()

In [ ]:
df.columns = ['ca', 'unit', 'scp', 'station', 'linename', 'division', 'date','time', 'desc', 'entries','exits']

In [ ]:
df['exits'].max()

In [ ]:
#df["date"] = pd.to_datetime(df["date"] + " " + df["time"])
df["date"] = pd.to_datetime(df["date"])
df.head()

### Counting daily traffic using `groupbby`

In [ ]:
df_entries = df.groupby(['station','ca','unit','scp','date']).entries.max() - df.groupby(['station','ca','unit','scp','date']).entries.min()
df_exits = df.groupby(['station','ca','unit','scp','date']).exits.max() - df.groupby(['station','ca','unit','scp','date']).exits.min()
print("Types: ", type(df_entries), type(df_exits))

In [ ]:
df_entries

We are now going to merge `df_entries` and `df_exits`. However, they need to have the same index.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [ ]:
df_entries_flat = df_entries.reset_index()
df_exits_flat = df_exits.reset_index()

df_traffic = pd.merge(df_entries_flat, df_exits_flat, how='outer')

df_traffic['day'] = df_traffic['date'].dt.day_name()
df_traffic['traffic'] = df_traffic['entries'] + df_traffic['exits']

In [ ]:
df_traffic.head()

In [ ]:
df_traffic.describe()

In [ ]:
df_traffic[["entries", "exits"]].hist();

In [ ]:
thresholdMax = 25000
df_traffic = df_traffic[df_traffic["traffic"] < thresholdMax]

In [ ]:
df_traffic[["entries", "exits", "traffic"]].hist();

Final steps

In [ ]:
df_station = df_traffic.groupby(['station','date','day']).sum().reset_index()
df_station.head()

### What happens at Penn station?

In [ ]:
df_station["station"].unique()

In [ ]:
df_Penn = df_station[df_station["station"] == "42 ST-PORT AUTH"]
df_Penn
df_Penn.plot.bar(x="day", y="traffic");

### Exercise

- What are the 3 busiest stations on average?
- What is the traffic distribution over the week?
- What are the busiest stations on weekends and on weekdays?

### Exercise

Write a script that imports each weekly file http://web.mta.info/developers/turnstile.html and merges them.

# Common issues

In [ ]:
xx = [2, 4, 6, '8']
df = pd.DataFrame(xx)
df.columns = ["X"]
df.head()

In [ ]:
df["Mult"] = 2.*df["X"]

In [ ]:
df.info()

In [ ]:
df['X'] = pd.to_numeric(df['X'])
df["Mult"] = 2.*df["X"]

In [ ]:
df.info()

In [ ]:
df.head()

### Working with copies

In [ ]:
N = 5
xx = np.random.standard_normal((N, 5))
df = pd.DataFrame(xx)
df2 = df ### same as df.copy(deep=False)
df3 = df.copy(deep=True)

In [ ]:
i,j = 0,1
df.iat[i,j] = 2.
print(df.iat[i,j], df2.iat[i,j])

In [ ]:
i,j = 0,1
df3.iat[i,j] = 5.
print(df.iat[i,j], df3.iat[i,j])

### Filling the void

In [ ]:
df = pd.read_csv("eurgbp-1m.csv", sep=";")#,header = None)
df.columns=["Date", "Time", "Bid", "Ask", "na", "na2", "na3"]
#df["DateTime"] = df["Date"] + df["Time"]
df = df.drop(columns=['na', 'na2', 'na3'])
df.head()

**Exercise:** Create a dataframe with one value for the bid and one value for the ask for each day.
For example, using the maximum value of the day.

In [ ]:
import random
chosenDay = df.iloc[random.choice(range(len(df))), 0]
print("Chosen day: ", chosenDay)

In [ ]:
oneDay = df[df["Date"] == chosenDay]
oneDay.set_index("Time", inplace=True)
del oneDay["Date"]
oneDay.plot(title="eurgbp on " + chosenDay);
oneDay.head()

In [ ]:
nbMissingValues = len(oneDay)//3
N = len(oneDay)
listMissingValues = random.choices(range(N), k=nbMissingValues)
for l in listMissingValues:
    oneDay.iat[l, 0] = np.log(-1.)

In [ ]:
oneDay.head()

In [ ]:
oneDay["Bid"].plot()

How do we fill the gaps? 
- `dropna`?
- interpolation? But which one?

In [ ]:
oneDay["Bid"].interpolate(method='linear', axis=0).plot();

# Extra: Combining DataFrames and creating signals

In [ ]:
aapl = pdr.DataReader("AAPL", 'yahoo', '1980-01-01')#["Close"]
coke = pdr.DataReader("COKE", 'yahoo', '1980-01-01')#["Close"]

In [ ]:
aapl.tail()

In [ ]:
coke.tail()

In [ ]:
df = pd.merge(coke, aapl, on="Date")
df.tail()

In [ ]:
df = pd.merge(coke["Close"], aapl["Close"], on="Date")
df.columns = ["coke", "aapl"]
df.head()

In [ ]:
df.plot();

#### Normalised versions...

In [ ]:
coke0 = coke.iloc[0,0]
aapl0 = aapl.iloc[0,0]

coke_norm = coke / coke0
aapl_norm = aapl / aapl0
df_norm = pd.merge(coke_norm["Close"], aapl_norm["Close"], on="Date")
df_norm.columns = ["coke norm", "aapl norm"]
df_norm.plot();

In [ ]:
coke = pdr.DataReader("COKE", 'yahoo', '1980-01-01')

In [ ]:
coke["Returns"] = np.log(coke['Close'] / coke['Close'].shift(1))
coke['MovingVol'] = coke['Returns'].rolling(window=252).std() * np.sqrt(252.)
coke = coke[["Close", "Returns", "MovingVol"]]
coke = coke.dropna()
coke["MovingVol"].plot(title="Moving vol");

In [ ]:
maxVolAllowed = 0.4
coke.loc[coke['MovingVol'] > maxVolAllowed, 'Signal'] = 0.6#coke["MovingVol"].max()
coke[["Signal", "MovingVol"]].plot().legend(loc=3);

# WARNINGS: Data inconsistencies

- Missing data
- NaN, -1, ...
- Date format
- Blank spaces
- Formats: Int, float, ...